# Triggering with threshold on channel

This example shows how a signal can be captured using a level trigger.

**Note:**  
Voltage range of fast analog inputs on the Red Pitaya depends on gain setting that can be set by jumpers. HV setting is for input range to ±20V, while LV sets input range to ±1V. For more information please read the following [chapter](http://redpitaya.readthedocs.io/en/latest/doc/developerGuide/125-14/fastIO.html#analog-inputs). 

Create a loop back from fast outputs to the fast inputs as shown in the picture below.  
Make sure that *gain setting* is set to ±1V (LV).

![Fast loop back](../img/FastIOLoopBack.png "Example of the fast loop back.")

Now start generation of the signal from the [generator notebook](gen_sync_two_channel.ipynb). Execute all cells except the last one which will disable the outputs. It is recommended that you run it when you finish with this example.

In [ ]:
from redpitaya.overlay.mercury import mercury as overlay
fpga = overlay()

Make an instance of the oscilloscope, the first argument is the channel index in range [0,1]. The second argument is the voltage range in volts, should be either 1.0 (default if unspecified) or 20.0.

In [ ]:
osc0 = fpga.osc(0, 1.0)

User should specify:
1. Sample rate decimation is an integer in range `[1, 2**17] = [1, 131072]`.
2. Number of samples stored before and after trigger, for a forced trigger only post trigger samples are enough.
3. Trigger level voltage hysteresis and edge.

In [ ]:
# data rate decimation 
osc0.decimation = 1

# trigger timing [sample periods]
N = osc0.buffer_size
osc0.trigger_pre  = N//4 * 1
osc0.trigger_post = N//4 * 3

# trigger level hysteresis [V] and edge ['neg', 'pos']
osc0.level = [0.4, 0.5]
osc0.edge  = 'pos'

# trigger source is the level trigger from the same input
osc0.trig_src = fpga.trig_src["osc0"]

The oscilloscope should be reset and started.
A loop is checking if a hardware trigger has arrived and if the desired number of data samples was already stored in the buffer.

The same number of samples as the sum of pre/post trigger delays should be read from the buffer.
The data read is aligned to the last sample, which will be always present as the last sample in the returned array.

In [ ]:
# reset and start
osc0.reset()
osc0.start()
# wait for data
while (osc0.status_run()): pass
print ('triggered')

import matplotlib.pyplot as plt

# show only the part of the buffer requested by pre/post trigger timing
data = osc0.data(N)
plt.plot(data)
plt.title('DATA');
plt.show()